In [2]:

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
from transformers import pipeline


In [4]:
df = pd.read_csv("C:/Users/Varad/Downloads/archive/Training Dataset.csv")

In [5]:
docs = df.astype(str).apply(" ".join, axis=1).tolist()


In [6]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(docs, convert_to_numpy=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Varad\Documents\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Varad\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

c:\Users\Varad\Documents\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [7]:
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

In [8]:
def retrieve_docs(query, k=3):
    query_embedding = embedder.encode([query])
    _, indices = index.search(query_embedding, k)
    return [docs[i] for i in indices[0]]

In [10]:
qa_pipeline = pipeline("text-generation", model="gpt2")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:  27%|##6       | 199M/747M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [11]:
def answer_question(query):
    context = " ".join(retrieve_docs(query))
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    result = qa_pipeline(prompt, max_new_tokens=100)
    return result[0]['generated_text']

In [12]:
print(answer_question("What factors affect loan approval?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: LP001586 Male Yes 3+ Not Graduate No 3522 0.0 81.0 180.0 1.0 Rural N LP002115 Male Yes 3+ Not Graduate No 2647 1587.0 173.0 360.0 1.0 Rural N LP002517 Male Yes 1 Not Graduate No 2653 1500.0 113.0 180.0 0.0 Rural N

Question: What factors affect loan approval?
Answer:

The following factors may influence loan approval:

1. The number of students with a history of serious psychological illness, and/or the frequency of mental health problems.

2. The number of students with a history of serious mental illness, and/or the frequency of mental health problems.

3. The number of students with a history of serious mental illness, and/or the frequency of mental health problems.

4. The number of students with a history of
